In [2]:
import numpy as np, pandas as pd

In [3]:
data = pd.read_csv('./dress_rental_prices.csv')

In [4]:
data.isna().sum()

Unnamed: 0      0
ID              0
Name            2
Brand           0
Colour          0
Catagories    114
Price           0
dtype: int64

In [5]:
data = data.rename({'Catagories': 'Categories'}, axis=1)

In [6]:
data.isna().sum()

Unnamed: 0      0
ID              0
Name            2
Brand           0
Colour          0
Categories    114
Price           0
dtype: int64

In [7]:
data_cleaned = data.dropna()

In [8]:
data_cleaned['Colour'].unique()

array(['beige', 'black', 'blue', 'brown', 'copper', 'cream', 'gold',
       'green', 'grey', 'multi', 'navy', 'orange', 'phantom', 'pink',
       'purple', 'red', 'silver', 'tan', 'white', 'yellow'], dtype=object)

In [9]:
data_cleaned.head()

,Unnamed: 0,ID,Name,Brand,Colour,Categories,Price
0,0,74416,Runway stripe dress,Stella McCartney,beige,dresses,111
1,1,73815,Reformation Kourtney Dress,Reformation,beige,dresses,50
2,2,73801,Ivory Viola bridal dress,Ghost,beige,dresses,75
3,3,73718,Pasu Dress - Rhino Tusk,Coucoo,beige,dresses,37
4,4,73605,Ellen,RIXO,beige,dresses,47


In [10]:
from sklearn.preprocessing import OneHotEncoder


ohe_colour = OneHotEncoder().fit_transform(data_cleaned['Colour'].values.reshape(-1, 1)).toarray()
ohe_categories = OneHotEncoder().fit_transform(data_cleaned['Categories'].values.reshape(-1, 1)).toarray()

In [11]:
X = np.column_stack([ohe_categories, ohe_colour])
y = data_cleaned['Price'].values

In [12]:
from sklearn.model_selection import train_test_split


xtrain, xtest, ytrain, ytest = train_test_split(X, y)

In [13]:
X.shape

(29881, 5419)

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from tensorflow.keras.models import Sequential

In [15]:
model = Sequential(
    [
        Input((xtrain.shape[1], )),
        Dense(64, activation='tanh'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1, activation='linear')
    ]
)

In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │       346,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 349,505 (1.33 MB)

 Trainable params: 349,505 (1.33 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(optimizer='adam', loss=MeanSquaredLogarithmicError, metrics=['msle'])

In [18]:
model.fit(xtrain, ytrain, epochs=20)

Epoch 1/20
701/701 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 4.6294 - msle: 4.6294
Epoch 2/20
701/701 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.2152 - msle: 0.2152
Epoch 3/20
701/701 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.2040 - msle: 0.2040
Epoch 4/20
701/701 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.1975 - msle: 0.1975
Epoch 5/20
701/701 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.1935 - msle: 0.1935
Epoch 6/20
701/701 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.1839 - msle: 0.1839
Epoch 7/20
701/701 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1805 - msle: 0.1805
Epoch 8/20
701/701 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.1813 - msle: 0.1813
Epoch 9/20
701/701 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1800 - msle: 0.1800
Epoch 10/20
701/701 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1783 - msle: 0.1783
Epoch 11/20
701/701 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.1783 - msle: 0.1783
Epoch 12/20
701/701 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.1788 - msle: 0.1788
Epoch 13/20
7

In [19]:
from sklearn.metrics import r2_score

In [20]:
print(ytrain, model.predict(xtrain))
print(ytest, model.predict(xtest))

701/701 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
[62 62 27 ... 36 48 84] [[58.354507]
 [54.768208]
 [59.180145]
 ...
 [58.354507]
 [58.354507]
 [54.81927 ]]
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[74 65 77 ... 36 34 44] [[55.622734]
 [58.354507]
 [54.768208]
 ...
 [61.87073 ]
 [59.180145]
 [54.219475]]
